In [1]:
!pip install -q langchain_huggingface
!pip install -q pinecone
!pip install -q langchain_pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.3/524.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 5.3 MB/s eta 0:00:00


In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
import getpass
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from pinecone import ServerlessSpec
import pandas as pd
from uuid import uuid4
from langchain_core.documents import Document

In [ ]:
if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = ""

pinecone_api_key = ""

In [3]:
pc = Pinecone(api_key=pinecone_api_key)

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="Qwen/Qwen3-Embedding-0.6B")

In [5]:
index_name = "hackathon"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

index = pc.Index(index_name)

In [6]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [ ]:
df = pd.read_json(
    "hf://datasets/Josephgflowers/Finance-Instruct-500k/train.json", lines=True
)

In [8]:
docs = [
    Document(page_content=f"User: {row['user']}\nAssistant: {row['assistant']}")
    for _, row in df.iterrows()
]

In [ ]:
from uuid import uuid4
from tqdm import tqdm

# Set a reasonable batch size
BATCH_SIZE = 32  # you can tune this

# Re-split using RecursiveCharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
chunks = splitter.split_documents(docs)

# Create matching UUIDs for each chunk
uuids = [str(uuid4()) for _ in range(len(chunks))]

# Add in batches
for i in tqdm(range(0, len(chunks), BATCH_SIZE), desc="Uploading to Pinecone"):
    batch_chunks = chunks[i : i + BATCH_SIZE]
    batch_ids = uuids[i : i + BATCH_SIZE]
    vector_store.add_documents(documents=batch_chunks, ids=batch_ids)


Uploading to Pinecone:  13%|█▎        | 3649/27733 [2:28:14<16:52:20,  2.52s/it]